In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import date

In [ ]:
banknifty_df=pd.read_csv("../input/nifty-bank/NIFTY BANK.csv")
banknifty_df.head()

In [ ]:
banknifty_df.info()

In [ ]:
banknifty_df.describe()

In [ ]:
banknifty_df['Prev_close']=banknifty_df['Close'].shift(1) # Calculate previous close
banknifty_df.head(2)

In [ ]:
# Calculate Percentage
banknifty_df['percentage']=(banknifty_df['Close']-banknifty_df['Prev_close'])*100/banknifty_df['Prev_close'] 
banknifty_df['percentage']=banknifty_df['percentage'].round(2)
banknifty_df.head()

In [ ]:
# Histogram of Percentage column
plt.figure(figsize=(20,6))
sns.displot(banknifty_df['percentage'],bins=100)

In [ ]:
banknifty_df['percentage'].describe()

In [ ]:
# Calculate Absolute Percentage
banknifty_df['abs_percentage']=abs(banknifty_df['Close']-banknifty_df['Prev_close'])*100/banknifty_df['Prev_close'] # Calculate abs percentage
banknifty_df['abs_percentage']=banknifty_df['abs_percentage'].round(2)
banknifty_df.head()

In [ ]:
plt.figure(figsize=(12,6))
sns.displot(banknifty_df['abs_percentage'],bins=100)

In [ ]:
banknifty_df['abs_percentage'].describe()

In [ ]:
# split date to year,month,day and day of the week
banknifty_df['Date'] = pd.to_datetime(banknifty_df['Date'])

banknifty_df['Date'] = pd.to_datetime(banknifty_df['Date'])
banknifty_df['year'] = pd.DatetimeIndex(banknifty_df['Date']).year
banknifty_df['Month'] = pd.DatetimeIndex(banknifty_df['Date']).month
banknifty_df['Day'] = pd.DatetimeIndex(banknifty_df['Date']).day
banknifty_df['dayOfWeek'] = banknifty_df['Date'].dt.day_name()
banknifty_df.head()

In [ ]:
#Plot Nifty Bank Closing price
plt.figure(figsize=(12,6))
plt.plot(banknifty_df['Date'],banknifty_df['Close'], color = 'blue', label = 'CLOSE PRICES', alpha = 0.7)
plt.legend(loc = 'upper right')
plt.title('BANK NIFTY Close Proce')

In [ ]:
# Create new Dataframe with 2020 year data
banknifty2020_df=banknifty_df[banknifty_df['year']==2020].copy()
banknifty2020_df.reset_index(drop=True,inplace=True)
banknifty2020_df.head()

In [ ]:
# Simple Moving average
def sma(data, window):
    sma = data.rolling(window = window).mean()
    return sma

banknifty2020_df['sma_20'] = sma(banknifty2020_df['Close'], 20)
banknifty2020_df.head()

In [ ]:
# Bollinger band (BB)
def bb(data, sma, window):
    std = data.rolling(window = window).std()
    upper_bb = sma + std * 2
    lower_bb = sma - std * 2
    return upper_bb, lower_bb

banknifty2020_df['upper_bb'], banknifty2020_df['lower_bb'] = bb(banknifty2020_df['Close'], banknifty2020_df['sma_20'], 20)
banknifty2020_df.head(5)

In [ ]:
#plot BB

def plot_BB(date,Close, sma_20, upper_bb, lower_bb):
    plt.figure(figsize=(12,6))
    plt.plot(date,Close, color = 'blue', label = 'CLOSE PRICES', alpha = 0.7)
    plt.plot(date,sma_20, label = 'MIDDLE BB', linewidth = 1.5, color = 'grey')
    plt.plot(date,upper_bb,label = 'UPPER BB', linestyle = '--', linewidth = 1, color = 'black')
    plt.plot(date, lower_bb,label = 'LOWER BB', linestyle = '--', linewidth = 1, color = 'black')
    plt.legend(loc = 'upper right')
    plt.title('BANK NIFTY BOLLINGER BANDS')

plot_BB(banknifty2020_df['Date'], banknifty2020_df['Close'],banknifty2020_df['sma_20'],banknifty2020_df['upper_bb'],banknifty2020_df['lower_bb'])

In [ ]:
def implement_bb_strategy(data, lower_bb, upper_bb):
    buy_price = []
    sell_price = []
    bb_signal = []
    signal = 0
    for i in range(1,len(data)):
        if data[i-1] < lower_bb[i-1] and data[i] > lower_bb[i]:
            if signal != 1:
                buy_price.append(data[i])
                sell_price.append(np.nan)
                signal = 1
                bb_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
        elif data[i-1] > upper_bb[i-1] and data[i] < upper_bb[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(data[i])
                signal = -1
                bb_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            bb_signal.append(0)
    buy_price.append(np.nan)
    sell_price.append(np.nan)
    bb_signal.append(0)
            
    return buy_price, sell_price, bb_signal

buy_price, sell_price, bb_signal = implement_bb_strategy(banknifty2020_df['Close'], banknifty2020_df['lower_bb'], banknifty2020_df['upper_bb'])

In [ ]:
def plot_BB_pred(date,Close, sma_20, upper_bb, lower_bb):
    plt.figure(figsize=(12,6))
    plt.plot(date,Close, color = 'blue', label = 'CLOSE PRICES', alpha = 0.7)
    plt.plot(date,sma_20, label = 'MIDDLE BB', linewidth = 1.5, color = 'grey')
    plt.plot(date,upper_bb,label = 'UPPER BB', linestyle = '--', linewidth = 1, color = 'black')
    plt.plot(date, lower_bb,label = 'LOWER BB', linestyle = '--', linewidth = 1, color = 'black')
    plt.scatter(date, buy_price, marker = '^', color = 'green', label = 'BUY', s = 200)
    plt.scatter(date, sell_price, marker = 'v', color = 'red', label = 'SELL', s = 200)
    plt.legend(loc = 'upper right')
    plt.title('BANK NIFTY BOLLINGER BANDS')

plot_BB_pred(banknifty2020_df['Date'], banknifty2020_df['Close'],banknifty2020_df['sma_20'],banknifty2020_df['upper_bb'],banknifty2020_df['lower_bb'])

In [ ]:

def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = exp1 - exp2
    signal = macd.ewm(span = smooth, adjust = False).mean()
    hist = macd - signal
    return macd,signal,hist


macd,signal,hist = get_macd(banknifty2020_df['Close'], 26, 12, 9)
banknifty2020_df['macd']=macd
banknifty2020_df['signal']=signal
banknifty2020_df['hist']=hist


In [ ]:
def plot_macd(date,prices, macd, signal, hist):
    print(len(hist))
    plt.figure(figsize=(12,6))
    plt.plot(date,macd, color = 'blue', linewidth = 1.5, label = 'MACD')
    plt.plot(date,signal, color = 'red', linewidth = 1.5, label = 'SIGNAL')
    plt.bar(date, hist, color = 'blue', alpha=0.2)
    plt.title("MACD")
    plt.legend()

plot_macd(banknifty2020_df['Date'], banknifty2020_df['Close'],banknifty2020_df['macd'],banknifty2020_df['signal'],banknifty2020_df['hist'])

In [ ]:
def implement_MACD_strategy(close,macd, signal_macd):
    buy_price = []
    sell_price = []
    bb_signal = []
    signal = 0
    for i in range(1,len(macd)):
        if macd[i-1] < signal_macd[i-1] and macd[i] > signal_macd[i]:
            if signal != 1:
                buy_price.append(close[i])
                sell_price.append(np.nan)
                signal = 1
                bb_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
        elif macd[i-1] > signal_macd[i-1] and macd[i] < signal_macd[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(close[i])
                signal = -1
                bb_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                bb_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            bb_signal.append(0)
    buy_price.append(np.nan)
    sell_price.append(np.nan)
    bb_signal.append(0)

    return buy_price, sell_price, bb_signal

buy_price, sell_price, bb_signal = implement_MACD_strategy(banknifty2020_df['Close'], banknifty2020_df['macd'], banknifty2020_df['signal'])

In [ ]:
def plot_macd(date,Close, macd, signal, hist):
    plt.figure(figsize=(12,6))
    plt.plot(date,Close, color = 'blue', label = 'CLOSE PRICES', alpha = 0.7)
    plt.scatter(date, buy_price, marker = '^', color = 'green', label = 'BUY', s = 100)
    plt.scatter(date, sell_price, marker = 'v', color = 'red', label = 'SELL', s = 100)
    plt.title("MACD Prediction signal")
    plt.legend()

plot_macd(banknifty2020_df['Date'], banknifty2020_df['Close'],banknifty2020_df['macd'],banknifty2020_df['signal'],banknifty2020_df['hist'])

In [ ]:
# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
bankniftyClose_df=scaler.fit_transform(np.array(banknifty_df['Close']).reshape(-1,1))

In [ ]:
#splitting dataset into train and test split
training_size=int(len(bankniftyClose_df)*0.9)
test_size=len(bankniftyClose_df)-training_size
train_data,test_data=bankniftyClose_df[0:training_size,:],bankniftyClose_df[training_size:len(bankniftyClose_df),:1]

In [ ]:

def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
# Create the Stacked LSTM model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64,verbose=1)

In [ ]:
# Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
# Transformback to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
# Calculate RMSE performance metrics
from sklearn.metrics import mean_squared_error
print("Train data RMSE: ",mean_squared_error(y_train,train_predict, squared=False))
# Test Data RMSE
print("Test data RMSE: ",mean_squared_error(y_test,test_predict, squared=False))

In [ ]:
### Plotting 
# shift train predictions for plotting
plt.figure(figsize=(12,6))
look_back=100
trainPredictPlot = np.empty_like(bankniftyClose_df)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# shift test predictions for plotting
testPredictPlot = np.empty_like(bankniftyClose_df)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(bankniftyClose_df)-1, :] = test_predict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(bankniftyClose_df),label="Close price")
plt.plot(trainPredictPlot,label="Train")
plt.plot(testPredictPlot,label="Test")
plt.legend()
plt.show()